# <span style="font-family:Times New Roman">Code 5. Музыкальные рекомендации</span> 

Импортируем необходимые библиотеки

In [15]:
import pandas as pd
import numpy as np
import glob
import pickle
import IPython

from scipy.spatial.distance import cosine, cityblock, correlation, canberra, chebyshev, euclidean

from get_features import get_song_features_set_1, get_song_name

Импортируем необходимые списки

In [2]:
with open('ExtraData\\gtzan_track_names.pkl', 'rb') as f:
    gtzan_track_names = pickle.load(f)

* **Canberra Distance**: Метрика Канберры `canberra`
* **Chebyshev Distance**: Расстояние Чебышёва `chebyshev`
* **Manhattan Distance**: Расстояние городских кварталов, манхэттенское `cityblock`
* **Correlation Distance**: Корреляция по расстоянию `correlation`
* **Cosine Distance**: Косинусное расстояние `cosine`
* **Euclidean Distance**: Евклидово расстояние `euclidean`

* Метрика Канберры определяется как
$$d(u,v) = \sum_i{\frac{|u_i - v_i|}{|u_i|+|v_i|}}$$

* Расстояние Чебышёва определяется как
$$d(u,v) = \max_i|u_i - v_i|$$

* Расстояние городских кварталов определяется как
$$d(u,v) = \sum_i|u_i - v_i|$$

* Корреляция по расстоянию определяется как
$$d(u,v) = 1 - \frac{|u - \bar{u}|\cdot|v - \bar{v}|}{\|u - \bar{u}\|_2\|v - \bar{v}\|_2}$$

* Косинусное расстояние определяется как
$$d(u,v) = 1 - \frac{u\cdot v}{\|u\|_2\|v\|_2}$$

* Евклидово расстояние определяется как
$$d(u,v) = \|u-v\|_2$$

<!-- К Manhattan ближе всего Euclidean, дальше Chebyshev, потом вместе Cosine и Correlation, последнее - Canberra\
**Canberra – Cosine/Correlation – Manhattan/Euclidean/Chebyshev** -->

In [3]:
distances = [cosine, cityblock, correlation, canberra, chebyshev, euclidean]

distances_names = ['Cosine Distance', 'Manhattan Distance',  
                   'Correlation Distance', 'Canberra Distance', 
                   'Chebyshev Distance', 'Euclidean Distance']

Список аудиофайлов из набора `GTZAN` (1):

In [16]:
path_default = 'C:\\Users\\micha\\OneDrive\\Рабочий стол\\Курсовая\\genres_original\\' 
files_default = glob.glob(path_default + "*\\*.wav")

Список аудиофайлов, добавленных вручную (2):

In [17]:
path = 'C:\\Users\\micha\\OneDrive\\Рабочий стол\\Курсовая\\Music'
files = glob.glob(path + "*\\*.wav")

Словарь с названиями аудиофайлов и путями к ним:

In [18]:
path_name_dict = dict()
for path in files:
    path_name_dict[get_song_name(path)] = path
    
for i, path in enumerate(files_default):
    path_name_dict[list(gtzan_track_names.values())[i]] = path

## <span style="font-family:Times New Roman">Используемые функции:</span> 

**1\.** `add_features` – добавить новые трек(и)

Если в наборе нет композиции, то рассчитываем для нее характеристики и добавляем в набор `features_set_enlarged.csv`

In [7]:
def add_features(features, files_to_add):
    for file in files_to_add:
        name = get_song_name(file)
        if not name in features.index:
            print(name)
            records = get_song_features_set_1(file)
            records['genre'] = 'Unknown'
            features.loc[name] = records
            
    features.to_csv('Sets\\features_set_enlarged.csv')
    return features

**2\.** `delete_features` – удалить трек(и)

Удаляем композиции и их характеристики из набора `features_set_enlarged.csv`

In [8]:
def delete_features(features, files_to_delete):
    for file in files_to_delete:
        name = get_song_name(file)
        if name in features.index:
            features.drop(index=name, inplace=True)
            
    features.to_csv('Sets\\features_set_enlarged.csv')
    return features

**3\.** `get_distance` – рассчет расстояний

Рассчитываем расстояние типа `distance` одной композиций с остальными, находящимися в `df`

In [9]:
def get_distance(df, features, distance):
    dist = np.array([distance(A, features) for A in df.values]).round(5)
    return dist

**4\.** `get_distance` – вывод аудио-объекта

С помощью `IPython` выводим интерфейс с возможностью воспроизведения звукового файла

In [10]:
def listen_audio(track_name):
    print(track_name)
    IPython.display.display(IPython.display.Audio(path_name_dict[track_name]))

**4\.** `find_closest` – рекомендация на основе музыкальной "близости"


На основе музыкальных характеристик функция отбирает `n` наиболее близких к конкретной композиции аудиофайлов. Есть возможность передать параметр `sorted_distance`, определяющий тип расстояния, по которому будет проводиться сортировка. Также, есть возможность передать параметр `audio_output`, который при значении  `True` позволяет прослушать композиции из списка рекомендаций.

In [11]:
def find_closest(df, track_name, n=10, sorted_distance=None, audio_output=False):
    df = df.copy()
    final_df = df[['genre']].copy()
    df.drop(columns=['genre'], inplace=True)
    
    track_name = get_song_name(track_name)
    track_idx = np.where(df.index == track_name)[0][0]
    track_features = df.iloc[track_idx].values
    
    for i, distance in enumerate(distances):
        final_df[distances_names[i]] = get_distance(df, track_features, distance)
        final_df[distances_names[i]] = final_df[distances_names[i]].rank()
    
    if sorted_distance == None:
        final_df['Rank'] = final_df.drop(columns=['genre']).sum(1)
        sorted_distance = 'Rank' 
    
    final_df.drop(index=[track_name], inplace=True)
    final_df.sort_values(sorted_distance, ascending=True, inplace=True)
    
    if audio_output:
        listen_audio(track_name)
        print('_' * len(track_name))
        for track in final_df.head(n).index:
            listen_audio(track)

    return final_df.head(n)

## <span style="font-family:Times New Roman">Подбор рекомендаций</span> 
Загружаем данные из файла `features_set_enlarged.csv`, где хранятся файлы из наборов (1) и (2)

In [25]:
features = pd.read_csv('Sets\\features_set_enlarged.csv', index_col=[0])

In [26]:
features.shape

(1099, 23)

In [30]:
features.iloc[-7:]

,spectral_centroid_mean,spectral_centroid_std,spectral_rolloff_mean,spectral_rolloff_std,spectral_bandwidth_mean,spectral_bandwidth_std,spectral_flux_mean,spectral_flux_std,zero_crossings_mean,zero_crossings_std,...,mfcc_1_std,mfcc_2_mean,mfcc_2_std,mfcc_3_mean,mfcc_3_std,mfcc_4_mean,mfcc_4_std,mfcc_5_mean,mfcc_5_std,genre
Nirvana – Come As You Are.wav,2474.668922,659.607128,5315.818056,1466.892498,2526.429249,496.273741,1.111476,1.154441,0.110180,0.046021,...,99.446320,81.105202,29.781263,1.752125,23.487410,36.848774,15.000340,8.589679,10.315143,Unknown
Queen - Another One Bites The Dust.wav,2293.723541,1063.086967,4558.332935,2089.505858,2295.501358,646.230588,1.959408,3.537536,0.111625,0.081036,...,129.503006,90.927994,50.710899,-7.161055,38.195992,29.677502,23.904655,7.839751,16.931730,Unknown
Queen – Don t Stop Me Now.wav,3123.289848,853.717043,5982.772330,1412.974976,2587.507419,347.275689,1.321040,1.206144,0.172598,0.079406,...,97.056473,52.126919,30.243607,-18.115164,23.434286,25.737743,16.249634,4.785786,12.215717,Unknown
Машина времени – Поворот.wav,2372.028293,740.031912,5065.596323,1557.238450,2560.645165,464.045756,1.274116,1.698707,0.090955,0.047057,...,87.683746,80.378250,25.242746,0.636832,17.350782,29.432810,16.154690,7.620935,12.633163,Unknown
Муслим Магомаев – Лучший город Земли.wav,1951.150470,601.885701,3866.374047,1347.705064,2042.850856,533.338694,1.251057,1.020806,0.101820,0.038793,...,121.498619,102.824165,31.248932,-14.733639,19.932064,31.363176,13.779013,-5.088217,16.413363,Unknown
Павел Кашин – Барышня.wav,2902.450969,795.214207,6195.558893,1163.851443,2766.880215,287.846211,1.363239,1.491126,0.134912,0.083057,...,87.736237,65.057190,23.172899,4.281864,15.337262,26.544344,13.661721,3.377597,11.354586,Unknown
Сергей Трофимов – Ветер в голове.wav,2045.229478,712.637558,4313.863662,1436.883564,2247.862449,455.874634,1.642617,2.260282,0.085485,0.044973,...,87.028374,96.691628,30.204744,-5.944005,21.214643,39.032768,16.617081,4.931193,14.699624,Unknown


In [44]:
%%time
find_closest(df=features, 
             track_name='Машина времени – Поворот.wav', 
             n=15, 
#              sorted_distance='Canberra Distance',
             audio_output=False)

Wall time: 128 ms


,genre,Cosine Distance,Manhattan Distance,Correlation Distance,Canberra Distance,Chebyshev Distance,Euclidean Distance,Rank
No Doubt – Dont Speak.wav,Unknown,26.0,2.0,21.0,5.0,11.0,4.0,69.0
Modjo – Lady (Hear Me Tonight).wav,Unknown,35.0,6.0,34.0,15.0,7.0,7.0,104.0
Earth Wind and Fire – Sing a Song,Disco,9.5,9.0,9.0,16.0,44.0,17.0,104.5
Led Zeppelin – Night Flight,Rock,8.0,5.0,10.0,88.0,3.0,3.0,117.0
Dexter Gordon – Willow Weep For Me,Jazz,17.0,7.0,18.5,99.0,6.0,6.0,153.5
Nirvana – Come As You Are.wav,Unknown,49.0,12.0,51.5,2.0,33.0,13.0,160.5
David Christie – Saddle Up,Disco,24.0,8.0,26.5,114.0,4.0,5.0,181.5
Gregory Isaacs – John Law,Reggae,3.0,3.0,3.0,174.0,2.0,2.0,187.0
Ini Kamoze – Here Comes The Hot Stepper(remix),Reggae,32.5,46.0,36.5,35.0,13.0,29.0,192.0
Dennis Brown – It's Magic,Reggae,64.0,20.0,71.5,23.0,12.0,18.0,208.5


In [43]:
listen_audio('Ani DiFranco – Shy')

Ani DiFranco – Shy
